# **Import modules**

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

# **Load data**

In [ ]:
import os


from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
data="/content/drive/MyDrive/LEONI_2021-2022/clean_dataset.csv"

In [ ]:
dataset = pd.read_csv(data)

In [ ]:
dataset.head()

,Leoni Part Number,Contact System,Finish,Gender,Material Name,Strip Length [mm],Sealability,Tab Thickness [mm],Tab Width [mm],Wire Material,Wire Max. External Diameter [mm],Wire Min. External Diameter [mm],Wire Size Range From [mm²],Wire Size Range To [mm²],id
0,P00117177,MQS 0.64,Au,male,CuNiSi,3.817235,sealable,0.640000,0.64,Cu,1.600000,1.142954,0.25,0.35,2
1,P00072506,JPT 2.8,Au,female,CuNiSi,3.500000,sealable,0.800000,2.80,Cu,2.300000,1.183465,0.20,0.50,4
2,P00087589,MCON 1.2,Sn,female,CuSn,3.921776,sealable,0.665182,1.20,Cu,1.656134,1.152643,0.14,0.22,7
3,P00087605,MAK 2.8,Sn,female,CuCrSiTi,4.009365,sealable,0.800000,2.80,Cu,1.651835,1.169176,0.20,0.35,11
4,P00088453,DFK 40 4.8,Sn,female,CuCrSiTi,4.846647,sealable,0.800000,4.80,Cu,3.105324,2.305979,1.50,2.50,13


In [ ]:
l=dataset['Finish'].unique()
l

array(['Au', 'Sn', 'Ag', 'Ni+Sn', 'Ni', 'Au+Ni', 'Ag+Sn', 'Zn',
       'Au+Ni+Sn', 'CuSn', 'Ag+Ni', 'Sn+Pb', 'CuSnP', 'Au+Pd'],
      dtype=object)

In [ ]:
#dataset[dataset['Finish'] == "Au+Ni"]

In [ ]:
import plotly.express as px
px.histogram(dataset, x="Contact System")

In [ ]:
dataset.columns

Index(['Leoni Part Number', 'Contact System', 'Finish', 'Gender',
       'Material Name', 'Strip Length [mm]', 'Sealability',
       'Tab Thickness [mm]', 'Tab Width [mm]', 'Wire Material',
       'Wire Max. External Diameter [mm]', 'Wire Min. External Diameter [mm]',
       'Wire Size Range From [mm²]', 'Wire Size Range To [mm²]', 'id'],
      dtype='object')

# Feature selection

In [ ]:
#dataset['Material Name']=dataset['Material Name'].replace('Cu-Alloy', 'CuAlloy')

In [ ]:
dataset.head(50)

,Leoni Part Number,Contact System,Finish,Gender,Material Name,Strip Length [mm],Sealability,Tab Thickness [mm],Tab Width [mm],Wire Material,Wire Max. External Diameter [mm],Wire Min. External Diameter [mm],Wire Size Range From [mm²],Wire Size Range To [mm²],id
0,P00117177,MQS 0.64,Au,male,CuNiSi,3.817235,sealable,0.640000,0.640000,Cu,1.600000,1.142954,0.250000,0.350000,2
1,P00072506,JPT 2.8,Au,female,CuNiSi,3.500000,sealable,0.800000,2.800000,Cu,2.300000,1.183465,0.200000,0.500000,4
2,P00087589,MCON 1.2,Sn,female,CuSn,3.921776,sealable,0.665182,1.200000,Cu,1.656134,1.152643,0.140000,0.220000,7
3,P00087605,MAK 2.8,Sn,female,CuCrSiTi,4.009365,sealable,0.800000,2.800000,Cu,1.651835,1.169176,0.200000,0.350000,11
4,P00088453,DFK 40 4.8,Sn,female,CuCrSiTi,4.846647,sealable,0.800000,4.800000,Cu,3.105324,2.305979,1.500000,2.500000,13
5,P00100668,JT 2.8,Ag,female,CuZn,4.500000,unsealable,0.800000,2.800000,Cu,2.554247,1.598245,0.500000,1.500000,22
6,P00100670,JT 2.8,Ag,female,CuSn,4.500000,unsealable,0.800000,2.800000,Cu,3.179026,2.046341,1.000000,2.500000,23
7,P00072483,DIA 2.5,Au,female,CuNiSi,3.807334,unsealable,0.686039,1.467008,Cu,1.695666,1.172706,0.200000,0.400000,27
8,P00072484,DIA 2.5,Au,female,CuNiSi,4.037126,unsealable,0.715458,1.861352,Cu,2.106976,1.458176,0.500000,1.000000,28
9,P00072485,DIA 2.5,Au,female,CuNiSi,4.596812,unsealable,0.809092,3.095786,Cu,3.156983,2.050394,1.010000,2.500000,29


In [ ]:
dataset['Material Name'].unique()

array(['CuNiSi', 'CuSn', 'CuCrSiTi', 'CuZn', 'CuZnPb', 'CuBe', 'Cu-Alloy',
       'CuFe', 'CuSnP', 'Cu', 'CuSnZn', 'Various (Assembly)', 'CuSnSi',
       'CuNiSiMg', 'CuFe/CrNi', 'CuFeP', 'CuSnO', 'CuNiSn', 'CuSn/CrNi',
       'CuMg', 'CuTeSn/CuBe', 'CuZnSn', 'CuTeSn', 'CuNiPbP', 'CuFeP/CuBe',
       'Fe', 'CuNiSiP', 'CuSn/CuBe', 'CuPb', 'CuFeTiCo', 'FeNi', 'PA12'],
      dtype=object)

In [ ]:
dataset['Finish'].unique()

array(['Au', 'Sn', 'Ag', 'Ni+Sn', 'Ni', 'Au+Ni', 'Ag+Sn', 'Zn',
       'Au+Ni+Sn', 'CuSn', 'Ag+Ni', 'Sn+Pb', 'CuSnP', 'Au+Pd'],
      dtype=object)

In [ ]:
#dataset['Finish']=dataset.Finish.replace('Ni+Sn', 'NiavecSn')

In [ ]:
#dataset['Finish']=dataset.Finish.replace('Au+Ni', 'AuavecNi')
#dataset['Finish']=dataset.Finish.replace('Ag+Sn', 'AgavecSn')
#dataset['Finish']=dataset.Finish.replace('Ag+Ni', 'AgavecNi')
#dataset['Finish']=dataset.Finish.replace('Sn+Pb', 'SnavecPb')
#dataset['Finish']=dataset.Finish.replace('Au+Pd', 'AuavecPd')

In [ ]:
#df['Contact System']=df['Contact System'].map(lambda x : x.lower())

In [ ]:
dataset['Finish']=dataset.Finish.replace('CUSNP', 'CuSnP')

In [ ]:
dataset.drop(dataset[dataset['Finish']=="blank"].index,inplace=True)

In [ ]:
#dataset['Finish']=dataset.Finish.replace('sn', 'Sn')

# **filter contact system**

In [ ]:
n = input("please enter your contact system")

please enter your contact systemmqs 0.64


In [ ]:
n=n.lower()
n

'mqs 0.64'

In [ ]:
dataset['Contact System']=dataset['Contact System'].map(lambda x : x.lower())

In [ ]:
df = dataset[dataset['Contact System']== n ]

In [ ]:
df.head()

,Leoni Part Number,Contact System,Finish,Gender,Material Name,Strip Length [mm],Sealability,Tab Thickness [mm],Tab Width [mm],Wire Material,Wire Max. External Diameter [mm],Wire Min. External Diameter [mm],Wire Size Range From [mm²],Wire Size Range To [mm²],id
0,P00117177,mqs 0.64,Au,male,CuNiSi,3.817235,sealable,0.64,0.64,Cu,1.600000,1.142954,0.25,0.35,2
24,P00072169,mqs 0.64,Au,female,CuNiSi,3.500000,unsealable,0.64,0.64,Cu,1.768718,1.194998,0.20,0.50,77
26,P00072165,mqs 0.64,Au,female,CuNiSi,3.500000,unsealable,0.64,0.64,Cu,1.768644,1.195047,0.20,0.50,215
27,P00117178,mqs 0.64,Au,male,CuNiSi,3.957289,unsealable,0.64,0.64,Cu,1.900000,1.370154,0.50,0.75,217
28,P00117179,mqs 0.64,Au,male,CuNiSi,3.791847,unsealable,0.64,0.64,Cu,1.600000,1.154537,0.25,0.35,218


In [ ]:
df.shape

(110, 15)

# **Finish**

In [ ]:
n = input("please enter Finish")

please enter FinishAu


In [ ]:
df.reset_index(inplace=True)

In [ ]:
df.shape

(110, 16)

In [ ]:
df.head(10)

,index,Leoni Part Number,Contact System,Finish,Gender,Material Name,Strip Length [mm],Sealability,Tab Thickness [mm],Tab Width [mm],Wire Material,Wire Max. External Diameter [mm],Wire Min. External Diameter [mm],Wire Size Range From [mm²],Wire Size Range To [mm²],id
0,0,P00117177,mqs 0.64,Au,male,CuNiSi,3.817235,sealable,0.640000,0.64,Cu,1.600000,1.142954,0.25,0.35,2
1,24,P00072169,mqs 0.64,Au,female,CuNiSi,3.500000,unsealable,0.640000,0.64,Cu,1.768718,1.194998,0.20,0.50,77
2,26,P00072165,mqs 0.64,Au,female,CuNiSi,3.500000,unsealable,0.640000,0.64,Cu,1.768644,1.195047,0.20,0.50,215
3,27,P00117178,mqs 0.64,Au,male,CuNiSi,3.957289,unsealable,0.640000,0.64,Cu,1.900000,1.370154,0.50,0.75,217
4,28,P00117179,mqs 0.64,Au,male,CuNiSi,3.791847,unsealable,0.640000,0.64,Cu,1.600000,1.154537,0.25,0.35,218
5,32,P00117112,mqs 0.64,Sn,female,CuNiSi,3.959699,sealable,0.640000,0.64,Cu,1.600000,1.219735,0.25,0.35,252
6,33,P00145673,mqs 0.64,Au,female,CuNiSi,3.350000,sealable,0.640000,0.64,Cu,1.900000,1.391066,0.50,0.75,262
7,257,P00130436,mqs 0.64,Sn,male,CuNiSi,3.050000,sealable,0.630000,0.63,Cu,1.250000,0.850000,0.13,0.17,848
8,261,P00072678,mqs 0.64,Au,female,CuNiSi,3.977631,sealable,0.640000,0.64,Cu,1.925035,1.391277,0.50,0.75,860
9,280,P00146977,mqs 0.64,Au,female,CuNiSi,3.500000,sealable,0.668484,0.64,Cu,1.900000,1.300000,0.50,0.75,922


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

scores = set()
tf = TfidfVectorizer()
tfidf_matrix = tf.fit_transform(df["Finish"])

cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

def index_from_product(df,product):

    return df[df['Finish']==product].index.values[0]
def recommenda( product, df,cosine_similarity_matrix):

    index = index_from_product(df,product)

    similarity_scores = list(enumerate(cosine_similarity_matrix[index]))
    
    similarity_scores_sorted = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    recommendations_indices = []
    for t in similarity_scores_sorted:
        recommendations_indices.append([t[0],t[1]])
        scores.add(t[1])

    l= sorted(scores)
    n= len(l)
    print(l[int(n/2)])
    lis=[]
    for i in recommendations_indices:
        if (i[1] != 0):
            id=df['id'].iloc[i[0]]
            lis.append(df[df["id"]==id].index[0])
            print("Recommended: " + df['Finish'].iloc[i[0]] + " (score:" + str(i[1]) + ")")
    print(lis)
    return(lis)

In [ ]:
l=recommenda(n, df, cosine_similarities)

1.0
Recommended: Au (score:1.0)
Recommended: Au (score:1.0)
Recommended: Au (score:1.0)
Recommended: Au (score:1.0)
Recommended: Au (score:1.0)
Recommended: Au (score:1.0)
Recommended: Au (score:1.0)
Recommended: Au (score:1.0)
Recommended: Au (score:1.0)
Recommended: Au (score:1.0)
Recommended: Au (score:1.0)
Recommended: Au (score:1.0)
Recommended: Au (score:1.0)
Recommended: Au (score:1.0)
Recommended: Au (score:1.0)
Recommended: Au (score:1.0)
Recommended: Au (score:1.0)
Recommended: Au (score:1.0)
Recommended: Au (score:1.0)
Recommended: Au (score:1.0)
Recommended: Au (score:1.0)
Recommended: Au (score:1.0)
Recommended: Au (score:1.0)
Recommended: Au (score:1.0)
Recommended: Au (score:1.0)
Recommended: Au (score:1.0)
Recommended: Au (score:1.0)
Recommended: Au (score:1.0)
Recommended: Au (score:1.0)
Recommended: Au (score:1.0)
Recommended: Au (score:1.0)
Recommended: Au (score:1.0)
Recommended: Au (score:1.0)
Recommended: Au (score:1.0)
Recommended: Au (score:1.0)
Recommended: Au 

In [ ]:
df_result= df.loc[l]

In [ ]:
df_result.shape


(53, 16)

In [ ]:
df_result.head()

,index,Leoni Part Number,Contact System,Finish,Gender,Material Name,Strip Length [mm],Sealability,Tab Thickness [mm],Tab Width [mm],Wire Material,Wire Max. External Diameter [mm],Wire Min. External Diameter [mm],Wire Size Range From [mm²],Wire Size Range To [mm²],id
0,0,P00117177,mqs 0.64,Au,male,CuNiSi,3.817235,sealable,0.64,0.64,Cu,1.600000,1.142954,0.25,0.35,2
1,24,P00072169,mqs 0.64,Au,female,CuNiSi,3.500000,unsealable,0.64,0.64,Cu,1.768718,1.194998,0.20,0.50,77
2,26,P00072165,mqs 0.64,Au,female,CuNiSi,3.500000,unsealable,0.64,0.64,Cu,1.768644,1.195047,0.20,0.50,215
3,27,P00117178,mqs 0.64,Au,male,CuNiSi,3.957289,unsealable,0.64,0.64,Cu,1.900000,1.370154,0.50,0.75,217
4,28,P00117179,mqs 0.64,Au,male,CuNiSi,3.791847,unsealable,0.64,0.64,Cu,1.600000,1.154537,0.25,0.35,218


# **Gender**

In [ ]:
gender = input("please enter the Gender") 

please enter the Gendermale


In [ ]:
if gender == 'male':
    df_male = df_result[df_result['Gender']== 'male' ]
    df_result=df_male

In [ ]:
if gender == 'female':
    df_female = df_result[df_result['Gender']== 'female' ]
    df_result= df_female

In [ ]:
if gender == 'hybrid':
    df_hybrid = df_result[df_result['Gender']== 'hybrid' ]
    df_result = df_hybrid

In [ ]:
df_result.head()

,index,Leoni Part Number,Contact System,Finish,Gender,Material Name,Strip Length [mm],Sealability,Tab Thickness [mm],Tab Width [mm],Wire Material,Wire Max. External Diameter [mm],Wire Min. External Diameter [mm],Wire Size Range From [mm²],Wire Size Range To [mm²],id
0,0,P00117177,mqs 0.64,Au,male,CuNiSi,3.817235,sealable,0.64,0.64,Cu,1.600000,1.142954,0.25,0.35,2
3,27,P00117178,mqs 0.64,Au,male,CuNiSi,3.957289,unsealable,0.64,0.64,Cu,1.900000,1.370154,0.50,0.75,217
4,28,P00117179,mqs 0.64,Au,male,CuNiSi,3.791847,unsealable,0.64,0.64,Cu,1.600000,1.154537,0.25,0.35,218
10,338,P00072367,mqs 0.64,Au,male,CuNiSi,3.500000,unsealable,0.64,0.64,Cu,1.543865,1.070187,0.14,0.22,996
11,360,P00087503,mqs 0.64,Au,male,CuNiSi,3.800000,sealable,0.64,0.64,Cu,1.899770,1.358937,0.50,0.75,1037


In [ ]:
df_result.shape

(17, 16)

In [ ]:
df_result['index']

0        0
3       27
4       28
10     338
11     360
12     361
16     583
31    1048
32    1049
38    1056
47    1267
59    1917
63    1951
79    3046
89    3436
90    3467
99    4353
Name: index, dtype: int64

In [ ]:
df_result.drop(['index'],axis = 1,inplace=True)

In [ ]:
df_result.head()

,Leoni Part Number,Contact System,Finish,Gender,Material Name,Strip Length [mm],Sealability,Tab Thickness [mm],Tab Width [mm],Wire Material,Wire Max. External Diameter [mm],Wire Min. External Diameter [mm],Wire Size Range From [mm²],Wire Size Range To [mm²],id
0,P00117177,mqs 0.64,Au,male,CuNiSi,3.817235,sealable,0.64,0.64,Cu,1.600000,1.142954,0.25,0.35,2
3,P00117178,mqs 0.64,Au,male,CuNiSi,3.957289,unsealable,0.64,0.64,Cu,1.900000,1.370154,0.50,0.75,217
4,P00117179,mqs 0.64,Au,male,CuNiSi,3.791847,unsealable,0.64,0.64,Cu,1.600000,1.154537,0.25,0.35,218
10,P00072367,mqs 0.64,Au,male,CuNiSi,3.500000,unsealable,0.64,0.64,Cu,1.543865,1.070187,0.14,0.22,996
11,P00087503,mqs 0.64,Au,male,CuNiSi,3.800000,sealable,0.64,0.64,Cu,1.899770,1.358937,0.50,0.75,1037


In [ ]:
df_result.reset_index(inplace=True)

# **Material Name**

In [ ]:
n = input("please enter Material Name")

please enter Material NameCuNiSi


In [ ]:
df_result.columns

Index(['index', 'Leoni Part Number', 'Contact System', 'Finish', 'Gender',
       'Material Name', 'Strip Length [mm]', 'Sealability',
       'Tab Thickness [mm]', 'Tab Width [mm]', 'Wire Material',
       'Wire Max. External Diameter [mm]', 'Wire Min. External Diameter [mm]',
       'Wire Size Range From [mm²]', 'Wire Size Range To [mm²]', 'id'],
      dtype='object')

In [ ]:
df_result['index']

0      0
1      3
2      4
3     10
4     11
5     12
6     16
7     31
8     32
9     38
10    47
11    59
12    63
13    79
14    89
15    90
16    99
Name: index, dtype: int64

In [ ]:
df_result.drop(['index'],axis = 1,inplace=True)

In [ ]:
df_result.head()

,Leoni Part Number,Contact System,Finish,Gender,Material Name,Strip Length [mm],Sealability,Tab Thickness [mm],Tab Width [mm],Wire Material,Wire Max. External Diameter [mm],Wire Min. External Diameter [mm],Wire Size Range From [mm²],Wire Size Range To [mm²],id
0,P00117177,mqs 0.64,Au,male,CuNiSi,3.817235,sealable,0.64,0.64,Cu,1.600000,1.142954,0.25,0.35,2
1,P00117178,mqs 0.64,Au,male,CuNiSi,3.957289,unsealable,0.64,0.64,Cu,1.900000,1.370154,0.50,0.75,217
2,P00117179,mqs 0.64,Au,male,CuNiSi,3.791847,unsealable,0.64,0.64,Cu,1.600000,1.154537,0.25,0.35,218
3,P00072367,mqs 0.64,Au,male,CuNiSi,3.500000,unsealable,0.64,0.64,Cu,1.543865,1.070187,0.14,0.22,996
4,P00087503,mqs 0.64,Au,male,CuNiSi,3.800000,sealable,0.64,0.64,Cu,1.899770,1.358937,0.50,0.75,1037


In [ ]:
df_result.reset_index(inplace=True)

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

scores = set()
tf = TfidfVectorizer()
tfidf_matrix = tf.fit_transform(df_result["Material Name"])

cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

def index_from_product(df,product):

    return df[df['Material Name']==product].index.values[0]
def recommenda( product, df,cosine_similarity_matrix):

    index = index_from_product(df,product)

    similarity_scores = list(enumerate(cosine_similarity_matrix[index]))
    
    similarity_scores_sorted = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    recommendations_indices = []
    for t in similarity_scores_sorted:
        recommendations_indices.append([t[0],t[1]])
        scores.add(t[1])

    l= sorted(scores)
    n= len(l)
    print(l[int(n/2)])
    lis=[]
    for i in recommendations_indices:
        if (i[1] != 0):
            id=df['id'].iloc[i[0]]
            lis.append(df[df["id"]==id].index[0])
            print("Recommended: " + df['Material Name'].iloc[i[0]] + " (score:" + str(i[1]) + ")")
    print(lis)
    return(lis)

In [ ]:
l=recommenda(n, df_result, cosine_similarities)

1.0
Recommended: CuNiSi (score:1.0)
Recommended: CuNiSi (score:1.0)
Recommended: CuNiSi (score:1.0)
Recommended: CuNiSi (score:1.0)
Recommended: CuNiSi (score:1.0)
Recommended: CuNiSi (score:1.0)
Recommended: CuNiSi (score:1.0)
Recommended: CuNiSi (score:1.0)
Recommended: CuNiSi (score:1.0)
Recommended: CuNiSi (score:1.0)
Recommended: CuNiSi (score:1.0)
Recommended: CuNiSi (score:1.0)
Recommended: CuNiSi (score:1.0)
Recommended: CuNiSi (score:1.0)
Recommended: CuNiSi (score:1.0)
Recommended: CuNiSi (score:1.0)
Recommended: CuNiSi (score:1.0)
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]


In [ ]:
df_result1= df_result.loc[l]

In [ ]:
df_result1.head()

,index,Leoni Part Number,Contact System,Finish,Gender,Material Name,Strip Length [mm],Sealability,Tab Thickness [mm],Tab Width [mm],Wire Material,Wire Max. External Diameter [mm],Wire Min. External Diameter [mm],Wire Size Range From [mm²],Wire Size Range To [mm²],id
0,0,P00117177,mqs 0.64,Au,male,CuNiSi,3.817235,sealable,0.64,0.64,Cu,1.600000,1.142954,0.25,0.35,2
1,1,P00117178,mqs 0.64,Au,male,CuNiSi,3.957289,unsealable,0.64,0.64,Cu,1.900000,1.370154,0.50,0.75,217
2,2,P00117179,mqs 0.64,Au,male,CuNiSi,3.791847,unsealable,0.64,0.64,Cu,1.600000,1.154537,0.25,0.35,218
3,3,P00072367,mqs 0.64,Au,male,CuNiSi,3.500000,unsealable,0.64,0.64,Cu,1.543865,1.070187,0.14,0.22,996
4,4,P00087503,mqs 0.64,Au,male,CuNiSi,3.800000,sealable,0.64,0.64,Cu,1.899770,1.358937,0.50,0.75,1037


# **Wire Size Range From [mm²]**

In [ ]:
number = float(input("enter the Wire Size Range From [mm²]"))

In [ ]:
df_result1['index']

In [ ]:
df_result1.drop(['index'],axis = 1,inplace=True)

In [ ]:
df_result1.reset_index(inplace=True)

In [ ]:
df_result1 ["Wire_Size_Range_From_[mm²]_diffrence"]=abs(number - df_result1["Wire Size Range From [mm²]"])

In [ ]:
df_result1["Wire_Size_Range_From_[mm²]_diffrence"].sort_values(ascending = True)

In [ ]:
len(df_result1)

In [ ]:

epsilon = float(input("enter the precison"))

In [ ]:
recommmended = []
recommmended.append(df_result1[df_result1['Wire_Size_Range_From_[mm²]_diffrence']< epsilon].index)
recommmended

In [ ]:
df_result2= df_result1.loc[recommmended[0]]

In [ ]:
df_result2.head()

# **Wire Size Range To [mm²]**

In [ ]:
number = float(input("enter the Wire Size Range To [mm²]"))

In [ ]:
df_result2['index']

In [ ]:
df_result2.drop(['index'],axis = 1,inplace=True)

In [ ]:
df_result2.reset_index(inplace=True)

In [ ]:
df_result2 ["Wire_Size_Range_To_[mm²]_diffrence"]=abs(number - df_result2["Wire Size Range To [mm²]"])

In [ ]:
df_result2["Wire_Size_Range_To_[mm²]_diffrence"].sort_values(ascending = True)

In [ ]:
epsilon = float(input("enter the precison"))

In [ ]:
recommmended = []
recommmended.append(df_result2[df_result2['Wire_Size_Range_To_[mm²]_diffrence']<= epsilon].index)
recommmended

In [ ]:
df_result3= df_result2.loc[recommmended[0]]

In [ ]:
df_result3.head()

# **Sealability**

In [ ]:
n = input("please enter sealability")

In [ ]:
df_result3.drop(['index'],axis = 1,inplace=True)

In [ ]:
df_result3.reset_index(inplace=True)

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

scores = set()
tf = TfidfVectorizer()
tfidf_matrix = tf.fit_transform(df_result3["Sealability"])

cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

def index_from_product(df,product):

    return df[df['Sealability']==product].index.values[0]
def recommenda( product, df,cosine_similarity_matrix):

    index = index_from_product(df,product)

    similarity_scores = list(enumerate(cosine_similarity_matrix[index]))
    
    similarity_scores_sorted = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    recommendations_indices = []
    for t in similarity_scores_sorted:
        recommendations_indices.append([t[0],t[1]])
        scores.add(t[1])

    l= sorted(scores)
    n= len(l)
    print(l[int(n/2)])
    lis=[]
    for i in recommendations_indices:
        if (i[1] != 0):
            id=df['id'].iloc[i[0]]
            lis.append(df[df["id"]==id].index[0])
            print("Recommended: " + df['Sealability'].iloc[i[0]] + " (score:" + str(i[1]) + ")")
    print(lis)
    return(lis)

In [ ]:
l=recommenda(n, df_result3, cosine_similarities)

In [ ]:
df_result4= df_result3.loc[l]

In [ ]:
df_result4.head()

# **Wire Material**

In [ ]:
n = input("please enter wire material")

In [ ]:
df_result4.drop(['index'],axis = 1,inplace=True)

In [ ]:
df_result4.reset_index(inplace=True)

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

scores = set()
tf = TfidfVectorizer()
tfidf_matrix = tf.fit_transform(df_result4["Wire Material"])

cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

def index_from_product(df,product):

    return df[df['Wire Material']==product].index.values[0]
def recommenda( product, df,cosine_similarity_matrix):

    index = index_from_product(df,product)

    similarity_scores = list(enumerate(cosine_similarity_matrix[index]))
    
    similarity_scores_sorted = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    recommendations_indices = []
    for t in similarity_scores_sorted:
        recommendations_indices.append([t[0],t[1]])
        scores.add(t[1])

    l= sorted(scores)
    n= len(l)
    print(l[int(n/2)])
    lis=[]
    for i in recommendations_indices:
        if (i[1] != 0):
            id=df['id'].iloc[i[0]]
            lis.append(df[df["id"]==id].index[0])
            print("Recommended: " + df['Wire Material'].iloc[i[0]] + " (score:" + str(i[1]) + ")")
    print(lis)
    return(lis)

In [ ]:
l=recommenda(n, df_result4, cosine_similarities)

In [ ]:
df_result5= df_result4.loc[l]

In [ ]:
df_result5.head()

# **Strip Length [mm]**

In [ ]:
number = float(input("enter the strip length"))

In [ ]:
df_result5['index']

In [ ]:
df_result5.drop(['index'],axis = 1,inplace=True)

In [ ]:
df_result5.reset_index(inplace=True)

In [ ]:
df_result5 ["Strip_Length_[mm]_diffrence"]=abs(number - df_result5["Strip Length [mm]"])
df_result5 ["Strip_Length_[mm]_diffrence"]

In [ ]:
df_result5["Strip_Length_[mm]_diffrence"].sort_values(ascending = True)

In [ ]:
epsilon = float(input("enter the precison"))

In [ ]:
recommmended = []
recommmended.append(df_result5[df_result5['Strip_Length_[mm]_diffrence']<= epsilon].index)
recommmended[0]

In [ ]:
df_result6= df_result5.loc[recommmended[0]]

In [ ]:
df_result6.head()

# **Tab Thickness [mm]**

In [ ]:
number = float(input("enter the Tab Thickness"))

In [ ]:
df_result6['index']

In [ ]:
df_result6.drop(['index'],axis = 1,inplace=True)

In [ ]:
df_result6.reset_index(inplace=True)

In [ ]:
df_result6 ["Tab_Thickness_[mm]_diffrence"]=abs(number - df_result6["Tab Thickness [mm]"])

In [ ]:
df_result6["Tab_Thickness_[mm]_diffrence"].sort_values(ascending = True)

In [ ]:
epsilon = float(input("enter the precison"))

In [ ]:
recommmended = []
recommmended.append(df_result6[df_result6['Tab_Thickness_[mm]_diffrence']< epsilon].index)
recommmended

In [ ]:
df_result7= df_result6.loc[recommmended[0]]

In [ ]:
df_result7.head()

# **Tab Width [mm]**

In [ ]:
number = float(input("enter the Tab Width"))

In [ ]:
df_result7['index']

In [ ]:
df_result7.drop(['index'],axis = 1,inplace=True)

In [ ]:
df_result7.reset_index(inplace=True)

In [ ]:
df_result7 ["Tab_Width_[mm]_diffrence"]=abs(number - df_result7["Tab Width [mm]"])

In [ ]:
df_result7["Tab_Width_[mm]_diffrence"].sort_values(ascending = True)

In [ ]:
epsilon = float(input("enter the precison"))

In [ ]:
recommmended = []
recommmended.append(df_result7[df_result7['Tab_Width_[mm]_diffrence']<= epsilon].index)
recommmended

In [ ]:
df_result8= df_result7.loc[recommmended[0]]

In [ ]:
df_result8.head()

# **Wire Max. External Diameter [mm]**

In [ ]:
number = float(input("enter the Wire Max. External Diameter [mm]"))

In [ ]:
df_result8['index']

In [ ]:
df_result8.drop(['index'],axis = 1,inplace=True)

In [ ]:
df_result8.reset_index(inplace=True)

In [ ]:
df_result8 ["Wire_Max._External_Diameter_[mm]_diffrence"]=abs(number - df_result8["Wire Max. External Diameter [mm]"])

In [ ]:
df_result8["Wire_Max._External_Diameter_[mm]_diffrence"].sort_values(ascending = True)

In [ ]:
epsilon = float(input("enter the precison"))

In [ ]:
recommmended = []
recommmended.append(df_result8[df_result8['Wire_Max._External_Diameter_[mm]_diffrence']< epsilon].index)
recommmended

In [ ]:
df_result9= df_result8.loc[recommmended[0]]

In [ ]:
df_result9.head()

# **Wire Min. External Diameter [mm]**

In [ ]:
number = float(input("enter the Tab Wire Min. External Diameter [mm]"))

In [ ]:
df_result9['index']

In [ ]:
df_result9.drop(['index'],axis = 1,inplace=True)

In [ ]:
df_result9.reset_index(inplace=True)

In [ ]:
df_result9 ["Wire_Min._External_Diameter_[mm]_diffrence"]=abs(number - df_result9["Wire Max. External Diameter [mm]"])

In [ ]:
df_result9["Wire_Min._External_Diameter_[mm]_diffrence"].sort_values(ascending = True)

In [ ]:
epsilon = float(input("enter the precison"))

In [ ]:
recommmended = []
recommmended.append(df_result9[df_result9['Wire_Min._External_Diameter_[mm]_diffrence']<= epsilon].index)
recommmended

In [ ]:
df_result10= df_result9.loc[recommmended[0]]

In [ ]:
df_result10.head()